In [55]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno
import plotly.express as px

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from xgboost import XGBRegressor

from sklearn.feature_selection import SelectKBest, f_regression

from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error, root_mean_squared_error
from sklearn.model_selection import cross_val_score

from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer, KNNImputer

from scipy import stats
import numpy as np
from scipy.stats import boxcox

# Importation des données

In [2]:
holidays = pd.read_csv("holidays.csv")
dates = pd.read_csv("dates.csv")
sample_submission = pd.read_csv("SampleSubmission.csv")
stores = pd.read_csv("stores.csv")
test = pd.read_csv("test.csv")
train = pd.read_csv("train.csv")

In [3]:
train_ = pd.merge(train, dates, how="left", on="date")
test_ = pd.merge(test, dates, how="left", on="date")
train__ = pd.merge(train_, holidays, on="date", how="left")
test__ = pd.merge(test_, holidays, on="date", how="left")
train___ = pd.merge(train__, stores, on="store_id", how='left')
test___ = pd.merge(test__, stores, on="store_id", how='left')
train___.rename(columns={'type_x': 'type_holidays', 'type_y': 'type_store'}, inplace=True)
test___.rename(columns={'type_x': 'type_holidays', 'type_y': 'type_store'}, inplace=True)

In [4]:
train___.head()

,date,store_id,category_id,target,onpromotion,nbr_of_transactions,year,month,dayofmonth,dayofweek,...,is_month_end,is_quarter_start,is_quarter_end,is_year_start,is_year_end,year_weekofyear,type_holidays,city,type_store,cluster
0,365,store_1,category_24,0.0,0,0.0,1,1,1,2,...,False,True,False,True,False,101,0.0,0,0,0
1,365,store_1,category_21,0.0,0,0.0,1,1,1,2,...,False,True,False,True,False,101,0.0,0,0,0
2,365,store_1,category_32,0.0,0,0.0,1,1,1,2,...,False,True,False,True,False,101,0.0,0,0,0
3,365,store_1,category_18,0.0,0,0.0,1,1,1,2,...,False,True,False,True,False,101,0.0,0,0,0
4,365,store_1,category_26,0.0,0,0.0,1,1,1,2,...,False,True,False,True,False,101,0.0,0,0,0


In [5]:
TARGET = "target"

In [6]:
test___["ID"] = (
    "year_week_" +
    test___["year_weekofyear"].astype(str) +
    "_" +
    test___["store_id"].astype(str) +
    "_" +
    test___["category_id"].astype(str)
)
train___.head()
train___["ID"] = (
    "year_week_" +
    train___["year_weekofyear"].astype(str) +
    "_" +
    train___["store_id"].astype(str) +
    "_" +
    train___["category_id"].astype(str)
)
train___.head()

,date,store_id,category_id,target,onpromotion,nbr_of_transactions,year,month,dayofmonth,dayofweek,...,is_quarter_start,is_quarter_end,is_year_start,is_year_end,year_weekofyear,type_holidays,city,type_store,cluster,ID
0,365,store_1,category_24,0.0,0,0.0,1,1,1,2,...,True,False,True,False,101,0.0,0,0,0,year_week_101_store_1_category_24
1,365,store_1,category_21,0.0,0,0.0,1,1,1,2,...,True,False,True,False,101,0.0,0,0,0,year_week_101_store_1_category_21
2,365,store_1,category_32,0.0,0,0.0,1,1,1,2,...,True,False,True,False,101,0.0,0,0,0,year_week_101_store_1_category_32
3,365,store_1,category_18,0.0,0,0.0,1,1,1,2,...,True,False,True,False,101,0.0,0,0,0,year_week_101_store_1_category_18
4,365,store_1,category_26,0.0,0,0.0,1,1,1,2,...,True,False,True,False,101,0.0,0,0,0,year_week_101_store_1_category_26


In [7]:
date_cols = dates.columns.to_list()
date_cols.remove('date')
quant_cols = [TARGET, "onpromotion"]
qual_cols = train___.drop(columns=[TARGET, "onpromotion", "nbr_of_transactions"]+date_cols+['store_id', 'category_id','ID', 'date']).columns.to_list()
print(quant_cols, qual_cols, date_cols)

['target', 'onpromotion'] ['type_holidays', 'city', 'type_store', 'cluster'] ['year', 'month', 'dayofmonth', 'dayofweek', 'dayofyear', 'weekofyear', 'quarter', 'is_month_start', 'is_month_end', 'is_quarter_start', 'is_quarter_end', 'is_year_start', 'is_year_end', 'year_weekofyear']


In [8]:
datas = train___.drop(columns=["store_id", "category_id", "ID", "date","nbr_of_transactions"])
test_data = test___.drop(columns=["store_id", "category_id", "ID", "date"])

In [9]:
test_data.head()

,onpromotion,year,month,dayofmonth,dayofweek,dayofyear,weekofyear,quarter,is_month_start,is_month_end,is_quarter_start,is_quarter_end,is_year_start,is_year_end,year_weekofyear,type_holidays,city,type_store,cluster
0,0,4,6,19,0,170,25,2,False,False,False,False,False,False,425,NaN,0,0,0
1,0,4,6,19,0,170,25,2,False,False,False,False,False,False,425,NaN,0,0,0
2,0,4,6,19,0,170,25,2,False,False,False,False,False,False,425,NaN,0,0,0
3,16,4,6,19,0,170,25,2,False,False,False,False,False,False,425,NaN,0,0,0
4,0,4,6,19,0,170,25,2,False,False,False,False,False,False,425,NaN,0,0,0


In [10]:
datas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2288088 entries, 0 to 2288087
Data columns (total 20 columns):
 #   Column            Dtype  
---  ------            -----  
 0   target            float64
 1   onpromotion       int64  
 2   year              int64  
 3   month             int64  
 4   dayofmonth        int64  
 5   dayofweek         int64  
 6   dayofyear         int64  
 7   weekofyear        int64  
 8   quarter           int64  
 9   is_month_start    bool   
 10  is_month_end      bool   
 11  is_quarter_start  bool   
 12  is_quarter_end    bool   
 13  is_year_start     bool   
 14  is_year_end       bool   
 15  year_weekofyear   int64  
 16  type_holidays     float64
 17  city              int64  
 18  type_store        int64  
 19  cluster           int64  
dtypes: bool(6), float64(2), int64(12)
memory usage: 257.5 MB


In [11]:
datas.isna().sum()

target                    0
onpromotion               0
year                      0
month                     0
dayofmonth                0
dayofweek                 0
dayofyear                 0
weekofyear                0
quarter                   0
is_month_start            0
is_month_end              0
is_quarter_start          0
is_quarter_end            0
is_year_start             0
is_year_end               0
year_weekofyear           0
type_holidays       1899612
city                      0
type_store                0
cluster                   0
dtype: int64

In [12]:
datas[quant_cols].describe()

,target,onpromotion
count,2.288088e+06,2.288088e+06
mean,3.942845e+02,3.151539e+00
std,1.179737e+03,1.346699e+01
min,0.000000e+00,0.000000e+00
25%,0.000000e+00,0.000000e+00
50%,1.500000e+01,0.000000e+00
75%,2.231933e+02,1.000000e+00
max,1.247170e+05,7.410000e+02


# EDA

## Analyse Univariée

In [13]:
# fig, axes = plt.subplots(1, 2, figsize=(5, 4))
# axes = axes.flatten()
# ax1 = axes[0]
# ax2 = axes[1]

# ax1.hist(datas[TARGET], bins=50)
# ax1.set_xlabel(TARGET)

# ax2.boxplot(datas[TARGET])
# ax2.set_xlabel(TARGET)
# plt.tight_layout()
# plt.show()

In [14]:
Q1 = datas[TARGET].quantile(0.25)
Q3 = datas[TARGET].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR
outliers = len(datas[TARGET][(datas[TARGET] < lower_bound) | (datas[TARGET] > upper_bound)])
perc = outliers/len(datas) * 100
print("{:.2f} % d'outliers".format(perc))

14.77 % d'outliers


nous remarquons une forte concentration des données autour de la moyenne et une grandes présences de outliers, nous allons gérer cela plus tard

In [15]:
# cols_to_plot = datas[quant_cols].drop(columns=["target"]).columns

# num_columns_to_plot = len(cols_to_plot)
# num_rows = num_columns_to_plot

# fig, axes = plt.subplots(num_rows, 2, figsize=(15, num_rows * 2.5))

# axes = axes.flatten()

# for i, col_name in enumerate(cols_to_plot):
#     hist_ax = axes[2 * i]
#     box_ax = axes[2 * i + 1]

#     hist_ax.hist(datas[col_name], bins=50, edgecolor='black', alpha=0.7)
#     hist_ax.set_title(f'Histogram of {col_name}')
#     hist_ax.set_xlabel(col_name)
#     hist_ax.set_ylabel('Frequency')

#     box_ax.boxplot(datas[col_name], vert=False, patch_artist=True, boxprops=dict(facecolor='skyblue'))
#     box_ax.set_title(f'Box Plot of {col_name}')
#     box_ax.set_xlabel(col_name)
#     box_ax.set_yticks([])

# plt.tight_layout()
# plt.show()

In [16]:
for col in quant_cols:
    Q1 = datas[col].quantile(0.25)
    Q3 = datas[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = len(datas[col][(datas[col] < lower_bound) | (datas[TARGET] > upper_bound)])
    perc = outliers/len(datas) * 100
    print("{:.2f} % d'outliers dans {}".format(perc, col))

14.77 % d'outliers dans target
66.29 % d'outliers dans onpromotion


il y a beaucoup d'outlier dans la colonne onpromotion, il seront géré plus tard

In [17]:
# for col in [TARGET, "onpromotion"]:
#     datas[col] = np.log1p(datas[col])

# test_data["onpromotion"] = np.log1p(test_data["onpromotion"])

In [18]:
datas.tail()

,target,onpromotion,year,month,dayofmonth,dayofweek,dayofyear,weekofyear,quarter,is_month_start,is_month_end,is_quarter_start,is_quarter_end,is_year_start,is_year_end,year_weekofyear,type_holidays,city,type_store,cluster
2288083,539.725,0,4,6,18,6,169,24,2,False,False,False,False,False,False,424,NaN,0,1,4
2288084,84.177,0,4,6,18,6,169,24,2,False,False,False,False,False,False,424,NaN,0,1,4
2288085,1973.760,3,4,6,18,6,169,24,2,False,False,False,False,False,False,424,NaN,0,1,4
2288086,2.000,0,4,6,18,6,169,24,2,False,False,False,False,False,False,424,NaN,0,1,4
2288087,27.076,0,4,6,18,6,169,24,2,False,False,False,False,False,False,424,NaN,0,1,4


In [19]:
datas["type_holidays"] = datas["type_holidays"].fillna(6)
test_data["type_holidays"] = test_data["type_holidays"].fillna(6)

## Analyse bivariée

In [20]:
# sns.pairplot(datas[quant_cols])

In [21]:
corr_matrix = datas[quant_cols].corr()
corr_matrix

,target,onpromotion
target,1.00000,0.43498
onpromotion,0.43498,1.00000


In [22]:
# sns.heatmap(corr_matrix, annot=True, cmap="coolwarm")

il existe une bonne relation entre target et onpromotion, cependant elle est plus faible entre nbr_of_transactions et target

In [23]:
qual_cols

['type_holidays', 'city', 'type_store', 'cluster']

In [24]:
# fig, axes = plt.subplots(2, 2, figsize=(8, 8))
# for i, col in enumerate(qual_cols):
#     ax = axes[i // 2, i % 2]
#     sns.boxplot(data=datas, x=col, y=TARGET, ax=ax)
#     ax.set_title(col)
# plt.tight_layout()
# plt.show()

les variables qualitatives type_holidays et type_store n'ont pas un impact significatif sur le target, elle seront supprimées dans le cadre de la construction de notre modèle.

#### Analyse en fonction des dates

In [25]:
len(date_cols)

14

In [26]:
# sns.lineplot(datas, x='date', y=TARGET)

In [27]:
# sns.lineplot(datas, x='year_weekofyear', y=TARGET)

In [28]:
# n_cols = 5  
# n_features = len(date_cols)
# n_rows = (n_features + n_cols - 1) // n_cols
# print(n_cols, n_rows)

# fig, axes = plt.subplots(n_rows, n_cols, figsize=(n_cols * 4, n_rows * 3)) 
# for i, col in enumerate(date_cols):
#     ax = axes[i // n_cols, i % n_cols]
#     sns.boxplot(datas, x=col, y=TARGET, ax=ax)
#     ax.set_title(col)
#     plt.xticks(rotation=90)
# plt.tight_layout()
# plt.show()

### Data Preprocessing

In [29]:
encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
encoded_features_array = encoder.fit_transform(datas[qual_cols])


datas_encoded = pd.DataFrame(encoded_features_array, columns=encoder.get_feature_names_out(qual_cols))

In [30]:

encoded_features_array_test = encoder.transform(test_data[qual_cols])
datas_test_encoded = pd.DataFrame(encoded_features_array_test, columns=encoder.get_feature_names_out(qual_cols))

In [31]:
encoded_cols = datas_encoded.columns.tolist()
datas[encoded_cols] = datas_encoded
test_data[encoded_cols] = datas_test_encoded

# modélisation

In [32]:
# preprocessor = ColumnTransformer(
#     transformers=[
#         ('num', 'passthrough', quant_cols),
#         ('bool', 'passthrough', qual_cols),   # Les colonnes booléennes passent (XGBoost les gérera comme 0/1)
#         ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_object_cols) # One-Hot pour les catégorielles
#     ])

In [33]:
# for col in date_cols_to_keep:
#     try:
#         date_cols.remove(col)
#     except:
#         pass

In [34]:
X = datas.drop(columns=[TARGET])
y = datas[TARGET]
X.columns

Index(['onpromotion', 'year', 'month', 'dayofmonth', 'dayofweek', 'dayofyear',
       'weekofyear', 'quarter', 'is_month_start', 'is_month_end',
       'is_quarter_start', 'is_quarter_end', 'is_year_start', 'is_year_end',
       'year_weekofyear', 'type_holidays', 'city', 'type_store', 'cluster',
       'type_holidays_0.0', 'type_holidays_1.0', 'type_holidays_2.0',
       'type_holidays_3.0', 'type_holidays_4.0', 'type_holidays_5.0',
       'type_holidays_6.0', 'city_0', 'city_1', 'city_2', 'city_3', 'city_4',
       'city_5', 'city_6', 'city_7', 'city_8', 'city_9', 'city_10', 'city_11',
       'city_12', 'city_13', 'city_14', 'city_15', 'city_16', 'city_17',
       'city_18', 'city_19', 'city_20', 'city_21', 'type_store_0',
       'type_store_1', 'type_store_2', 'type_store_3', 'type_store_4',
       'cluster_0', 'cluster_1', 'cluster_2', 'cluster_3', 'cluster_4',
       'cluster_5', 'cluster_6', 'cluster_7', 'cluster_8', 'cluster_9',
       'cluster_10', 'cluster_11', 'cluster_12', '

In [35]:
y = np.log1p(y)

In [36]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [50]:
selector = SelectKBest(score_func=f_regression, k=40)
X_train_selected = selector.fit_transform(X_train, y_train)
X_test_selected = selector.transform(X_test)
print(X_train_selected)

[[0. 2. 7. ... 0. 0. 0.]
 [0. 2. 6. ... 0. 0. 0.]
 [0. 1. 7. ... 0. 0. 0.]
 ...
 [5. 3. 7. ... 0. 0. 0.]
 [0. 4. 5. ... 0. 0. 0.]
 [7. 4. 5. ... 0. 0. 0.]]


In [53]:
params = {
    'objective': 'reg:squarederror',
    'eval_metric': 'rmse',
    'max_depth': 6,
    'learning_rate': 0.1,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'random_state': 42
}


params['reg_alpha'] = 0.1
params['reg_lambda'] = 0.1

In [54]:
model = XGBRegressor(params=params)
model.fit(X_train_selected, y_train)
y_pred = model.predict(X_test_selected)
rmse = root_mean_squared_error(y_test, y_pred)
print(rmse)

/home/hermann/miniconda3/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [18:50:19] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "params" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


1.9216540106456343


In [ ]:
# model = XGBRegressor()
# model.fit(X_train, y_train)
# y_pred = model.predict(X_test)
# rmse = root_mean_squared_error(y_test, y_pred)
# print(rmse)

1.920003374168876


# Submission

In [41]:
# test_with_drop = test_data.drop(columns=['type_holidays', 'type_store',])
# test_with_drop.head()

In [42]:
model.predict(X)
y_ = model.predict(test_data)

In [43]:
submission = pd.DataFrame()
submission['ID'] = test___["ID"]
submission['target'] = np.expm1(y_)
submission = submission.set_index("ID")

In [44]:
submission.to_csv("submission.csv")

In [49]:
print(X.head())

   onpromotion  year  month  dayofmonth  dayofweek  dayofyear  weekofyear  \
0            0     1      1           1          2          1           1   
1            0     1      1           1          2          1           1   
2            0     1      1           1          2          1           1   
3            0     1      1           1          2          1           1   
4            0     1      1           1          2          1           1   

   quarter  is_month_start  is_month_end  ...  cluster_7  cluster_8  \
0        1            True         False  ...        0.0        0.0   
1        1            True         False  ...        0.0        0.0   
2        1            True         False  ...        0.0        0.0   
3        1            True         False  ...        0.0        0.0   
4        1            True         False  ...        0.0        0.0   

   cluster_9  cluster_10  cluster_11  cluster_12  cluster_13  cluster_14  \
0        0.0         0.0         0